# Compte rendu numéro 2 -  08-03/15-03

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
os.chdir(sys.path[0]+"/..")
cwd = os.getcwd()
print(cwd)

In [ ]:
from added import *
from runwatermaze import perform_pearce

Suite à notre dernière discussion, on s'était dit qu'il fallait:

<ul> 
<li>Créer une carte des heading vectors, notamment afin de mieux comprendre la stratégie utilisée par la stratégie MF. (Et pourquoi elle semble fausser les résultats dans l'expérience de Pearce où les drapeaux sont inversés).
</li>
<br>
<li>Resimuler l'expérience principale de Pearce mais avec une distance plateforme-drapeau deux fois plus grande que celle utilisée par Geerts.
</li>
<br>
<li>Recommencer l'expérience principale de Pearce avec une stratégie model-free allocentrique et non plus égocentrique.
</li>
<br>
<li>
Réaliser une expérience où deux groupes d'agents sont soumis à une phase de test après entraînement sur 11 sessions de 4 trials du water-maze. Pour un des deux groupes, la position des drapeaux est inversée dans la phase de test.
</li>
<br>
</ul>

### Modification du labyrinthe utilisé par Geerts

Avant tout je me suis rendu compte en créant les heading-vectors et en manipulant les coordonnées réelles des plateformes et des drapeaux que la distance qui les sépare dans le code de Geerts n'est pas de 1 case mais en fait d'une demi case. Ci-dessous, le point bleu devrait se trouver un petit peu plus haut, là où pointe les trois flèches.

In [ ]:
plot_img("demi_distance.jpg", (8,8))

Ci-dessous les water-mazes utilisés par Pearce (gauche) et Geerts (droite). Les plateformes sont indiquées respectivement par les cercles vides et par les hexagones rouges. Les drapeaux sont indiqués respectivement par les cercles pleins noirs et les hexagones bleus. Les drapeaux ne sont pas indiqués sur le labyrinthe original de Geerts car ils sont trop proche de la plateforme (1/2 case).

Dans l'expérience de Pearce, la piscine fait 2 mêtres de diamêtre et il y a 20 cm d'espace entre le drapeau et la plateforme. Le diamêtre du Water-Maze numérique étant ici de 20 cases, la distance correcte entre drapeaux et plateformes est effectivement, comme tu le proposais, de 2 cases. Ca correspond donc à quatre fois la distance utilisée dans le code de Geerts

In [ ]:
plot_2img("pearce_maze.jpg", "geerts_maze.jpg", (14,6))

L'agencement des plateformes dans le labyrinthe de Geerts ne m'a pas permis pas de créer un drapeau distant de 2 cases vers le nord. J'ai donc rapproché les cases du centre (ce qui correspond d'ailleurs plus à la conformation originale de Pearce).

In [ ]:
plot_2img("pearce_maze.jpg", "my_geerts_maze.jpg", (14,6))

En plus grand si l'indice des cases t'intéresse.

In [ ]:
plot_geerts(10, platforms=[48, 52, 118, 122, 126, 94, 98, 44], landmarks=[108, 112, 116, 56, 60, 40, 100, 104], figsize=(4,4))

Pour rappel:

In [ ]:
plot_2img("results_pearce.jpg","geerts_exp_1.jpg", figsize=(16,16))

### Réplication de Pearce en utilisant une distance de 2 cases et la nouvelle topologie

In [ ]:
exp = "first_exp_pearce"
maze_size = 10
n_sessions = 11
n_trials = 4
n_agents = 100
mf_allo = False
sr_lr = 0.07
q_lr = 0.07
inv_temp = 16
gamma = 0.95
eta = 0.03 # reliability learning rate
mpe = 1 # maximum prediction error
A_alpha = 1.8 # Steepness of transition curve MF to SR 
A_beta = 1.1 # Steepness of transition curve SR to MF 
landmark_dist = 4

In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, n_agents, mf_allo, sr_lr, q_lr, inv_temp, gamma, eta, mpe, A_alpha, A_beta, landmark_dist)

Ci-dessus les performances de l'algorithme sur le water-maze de Pearce. C'est le meilleur fit que j'ai pu obtenir pour l'instant. Voilà ce qui change principalement avec les résultats précedents:

- La plateforme est globalement plus rapidement atteinte: c'est probablement dû au fait que les plateformes sont plus resserrées
<br><br>
- Le P(SR) final est bien plus élevé: lorsque la distance plateforme-landmark était de 0.5 cases, le P(SR) final était d'environ 0.2, celà est probablement dû aux mauvaises performances de la stratégie MF égocentrique lorsque le drapeau est loin de la plateforme.
<br><br>
- Les performances au premier essai ne rejoignent pas les performances aux essais suivants: celà est dû au fait que la stratégie MF égocentrée ne peut pas atteindre directement la plateforme sans collaboration avec la SR. Seule, elle doit adopter à proximité du drapeau une stratégie aléatoire (ci-dessus) ou en spirale (voir ci-dessous).

<b>Note:</b> Ici la reward-function des 100 agents n'a pas été mise à jour sur la plateforme 0. La SR indique donc les emplacements de plateformes précédentes.

Enfin les performances aux essais 2,3 et 4 sont moins bonnes que dans le compte-rendu précédent, c'est principalement du au fait que dans le code de Geerts, il y avait une phase d'apprentissage latent. Or il n'y a pas d'apprentissage latent dans l'expérience de Pearce. J'ai donc initialisé toutes les valeurs des place-cells à 0.


In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, 1, mf_allo, sr_lr, q_lr, 25, gamma, eta, mpe, A_alpha, A_beta, landmark_dist, show_perfs=False)

Ci-dessous, la reward-function des 10 agents a été mise à jour sur la plateforme 0. La SR pointe donc sur 0 et on voit en bas à droite la coopération entre SR et MF, qui est pertinente pour les deuxièmes, troisièmes et quatrièmes essais.

In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, 10, mf_allo, sr_lr, q_lr, 25, gamma, eta, mpe, A_alpha, A_beta, landmark_dist, show_perfs=False)

### Fonctionnement de la stratégie MF égocentrée

L'agent et son champ de vision sont placés au premier trial dans la direction du drapeau, puis à chaque nouveau pas ils sont placés dans le sens du vecteur allant de s à s+1.

Le vecteur comportant 80 features encodant le champ de vision est multiplié par les 6 actions égocentriques disponibles (tout droit, en avant à gauche, en arrière à gauche, en arrière... etc).

Les Q-values égocentriques ainsi formées sont utilisées en association avec l'orientation de l'agent pour former un vecteur de Q-values allocentriques, qui est fusionné au vecteur de Q-values issu de la Successor Representation pour la prise de décision finale.

Dans la partie suivante j'ai relancé les simulations précédentes avec une stratégie MF allocentrée(ique ?), où l'orientation de l'agent n'a pas été prise en compte lors des calculs.


### Réplication de Pearce en utilisant une stratégie MF allocentrée

In [ ]:
exp = "first_exp_pearce"
maze_size = 10
n_sessions = 11
n_trials = 4
n_agents = 100
mf_allo = True
sr_lr = 0.07
q_lr = 0.07
inv_temp = 16
gamma = 0.95
eta = 0.03 # reliability learning rate
mpe = 1 # maximum prediction error
A_alpha = 1.8 # Steepness of transition curve MF to SR 
A_beta = 1.1 # Steepness of transition curve SR to MF 
landmark_dist = 4

In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, n_agents, mf_allo, sr_lr, q_lr, inv_temp, gamma, eta, mpe, A_alpha, A_beta, landmark_dist)

Le fit est meileur, les performances au premier essai rejoignent à la fin de la simulation celles des trois autres essais.
La P(SR) finale est moins élevée qu'avec un MF égocentrique, probablement car la stratégie allocentrique à de meilleures performances. On voit dans les graphes montrant les heading vectors que l'agent adopte non plus une stratégie aléatoire ou en spirale à l'approche du drapeau, mais qu'il est capable d'identifier si il se trouve à l'ouest ou à l'est du drapeau et de toujours descendre en direction de la plateforme. Cela se fait même sans collaboration avec la SR.

<b>Note:</b> Le graphe des heading vectors égocentriques ne correspond plus à des actions égocentriques (je vais le supprimer pour ce cas là)

### Réplication de la seconde expérience de Pearce (changement de plateforme à chaque trial) avec MF égocentrée

In [ ]:
exp = "second_exp_pearce"
maze_size = 10
n_sessions = 11
n_trials = 4
n_agents = 100
mf_allo = False
sr_lr = 0.07
q_lr = 0.07
inv_temp = 16
gamma = 0.95
eta = 0.03 # reliability learning rate
mpe = 1 # maximum prediction error
A_alpha = 1.8 # Steepness of transition curve MF to SR 
A_beta = 1.1 # Steepness of transition curve SR to MF 
landmark_dist = 4

In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, n_agents, mf_allo, sr_lr, q_lr, inv_temp, gamma, eta, mpe, A_alpha, A_beta, landmark_dist)

### Réplication de la seconde expérience de Pearce (changement de plateforme à chaque trial) avec MF allocentrée

In [ ]:
exp = "second_exp_pearce"
maze_size = 10
n_sessions = 11
n_trials = 4
n_agents = 100
mf_allo = True
sr_lr = 0.07
q_lr = 0.07
inv_temp = 16
gamma = 0.95
eta = 0.03 # reliability learning rate
mpe = 1 # maximum prediction error
A_alpha = 1.8 # Steepness of transition curve MF to SR 
A_beta = 1.1 # Steepness of transition curve SR to MF 
landmark_dist = 4

In [ ]:
perform_pearce(exp, maze_size, n_trials, n_sessions, n_agents, mf_allo, sr_lr, q_lr, inv_temp, gamma, eta, mpe, A_alpha, A_beta, landmark_dist)

### Réplication de la troisième expérience de Pearce

Dans cette expérience, Pearce et al, ont soumis deux groupes de rats à une même phase d'entraînement: les 11 sessions * 4 essais de la tâche de Pearce. Chaque groupe a été soumi à une phase de test différent. 
<br><br>
Pour le groupe contrôle, le test a été un épisode classique du water-maze, avec pour la première fois la plateforme placée au centre, et le drapeau deux cases (ou 20cm) au nord de la plateforme. Pour le groupe expérimental, la plateforme est aussi placée au centre, mais le landmark est situé 2 cases au sud. 
<br><br>
Les résultats de Pearce montrent que les rats, qu'ils soient sains ou atteint de lésions hippocampiques complètent un épisode en environ 10 secondes lorsque le drapeau est dans une orientation similaire à l'entraînement. Dans la condition inverse, les rats mettent environ 60 secondes pour atteindre la plateforme.

Dans les cellules suivantes je simule l'expérience sur des agents sains uniquement.

<b>Note</b>: Le -4 en paramêtre fait référence à des demi-cases et non des cases entières (à modifier).

### Troisième expérience de Pearce avec stratégie MF allocentrique

In [ ]:
exp3_pearce(allo=True, agents_per_group=1000, alt_dist=-4)

<b>Important: </b> La SR n'a pas accès à la position de la plateforme pendant le test. Les graphes ont été générés après l'épisode de test, or la reward function liée à la SR à été mise à jour à la fin de cet épisode de test. C'est pour cette raison que la stratégie de la SR converge vers 0 dans les graphes ci-dessus.
<br><br>
Ci-dessus, les résultats sont congruents avec ceux de Pearce, les agents mettent plus de temps à atteindre la plateforme dans la condition inversée. Le ratio entre condition n'est pas correct cependant: 1/6 pour Pearce, et environ 1/25 pour Geerts-allocentrique. 

### Troisième expérience de Pearce avec stratégie MF égocentrique

In [ ]:
exp3_pearce(allo=False, agents_per_group=1000, alt_dist=-4)

Je n'ai pas réussi à répliquer les résultats dont je t'ai parlé la dernière fois, où le groupe inverse était le plus rapide, cela avait sûrement pour origine un trop petit échantillon d'agents. <br><br>
Ci-dessus, il n'y aucune différence entre les conditions normales et inverses avec un MF égocentrique, chaque situation est traitée de la même manière pourvu que la distance entre drapeau et agent soit similaire, cela se visualise par "l'effet spirale" à proximité du landmark.
<br><br><b>En conclusion, en plus de fournir de meilleures performances, il semble qu'une stratégie modèle-free allocentrique soit la seule a pouvoir expliquer les résultats expérimentaux de Pearce.</b> 
    
Reste à savoir comment celle-ci est implémentée. Est-ce que la méthode utilisée ici est celle utilisée par les rats ? C'est sûrement la méthode la plus parcimonieuse. Sinon on pourrait imaginer que les rats possèdent une représentation de la distance réelle entre drapeau et plateforme, qui ne reposerait pas directement sur la perception immédiate de la distance et de la direction du drapeau par l'agent.